# 0. Import, Loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('output/df_house_info_1.csv')

In [4]:
second = pd.read_csv('features/export-reestrmkd-39-20221101.csv', sep=';')

In [5]:
second_columns = [
    'management_organization_id',
    'built_year',
    'exploitation_start_year',
    'project_type',
    'house_type',
    'is_alarm',
    'method_of_forming_overhaul_fund',
    'floor_count_max',
    'floor_count_min',
    'entrance_count',
    'elevators_count',
    'energy_efficiency',
    'quarters_count',
    'living_quarters_count',
    'unliving_quarters_count',
    'area_total',
    'area_residential',
    'area_non_residential',
    'area_common_property',
    'area_land',
    'parking_square',
    'playground',
    'sportsground',
    'other_beautification',
    'foundation_type',
    'floor_type',
    'wall_material',
    'basement_area',
    'chute_type',
    'chute_count',
    'electrical_type',
    'electrical_entries_count',
    'heating_type',
    'hot_water_type',
    'cold_water_type',
    'sewerage_type',
    'sewerage_cesspools_volume',
    'gas_type',
    'ventilation_type',
    'firefighting_type',
    'drainage_type'
]

In [6]:
address_cols = [
    'formalname_city',
    'formalname_street',
    'house_number',
    'building',
    'block',
    'letter',
]

# 1. Street preprocessing

In [7]:
df['prep_street'] = df['street'].map(
    lambda x: " ".join(
        x.lower()
        .replace('улица', '')
        .replace('аллея', '')
        .replace('переулок', '')
        .replace('проспект', '')
        .replace('проезд', '')
        .replace('бульвар', '')
        .replace('шоссе', '')
        .replace('площадь', '')
        .replace('набережная', '')
        .replace('тупик', '')
        .split()
    )
    if not pd.isna(x)
    else x
)

In [8]:
second['formalname_street'] = second['formalname_street'].map(
    lambda x: " ".join(
        x.lower()
        .replace('улица', '')
        .replace('аллея', '')
        .replace('переулок', '')
        .replace('проспект', '')
        .replace('проезд', '')
        .replace('бульвар', '')
        .replace('шоссе', '')
        .replace('площадь', '')
        .replace('набережная', '')
        .replace('тупик', '')
        .split()
    )
    if not pd.isna(x)
    else x
)

In [9]:
second['formalname_street'] = second['formalname_street'].map(
    lambda x: " ".join(sorted(x.split(), key=lambda y: not('-' in y)))
    if not pd.isna(x)
    else x
)

In [10]:
street_maps = {
    'туруханская': '1-й туруханский',
    '3-я большая окружная': '3-я окружная',
    '4-я большая окружная': '4-я окружная',
    '9 апреля': '9-го апреля',
    'а.суворова': 'александра суворова',
    'а.толстого': 'алексея толстого',
    'б.хмельницкого': 'богдана хмельницкого',
    'в.гакуна': 'виктора гакуна',
    'в.денисова': 'виктора денисова',
    'в.талалихина': 'виктора талалихина',
    'г.димитрова': 'георгия димитрова',
    'о.кошевого': 'олега кошевого',
    'в.фермора': 'виллима фермора',
    'д.донского': 'дмитрия донского',
    'и.земнухова': 'ивана земнухова',
    'к.цеткин': 'клары цеткин',
    'л.голикова': 'лени голикова',
    'л.толстого': 'льва толстого',
    'м.светлова': 'михаила светлова',
    'п.морозова': 'павлика морозова',
    'р.люксембург': 'розы люксембург',
    'с.тюленина': 'сергея тюленина',
    'с.разина': 'степана разина',
    'с.халтурина': 'степана халтурина',
    'ф.воейкова': 'федора воейкова',
    'ф.лефорта': 'франца лефорта',
    'ю.гагарина': 'юрия гагарина',
    'ю.маточкина': 'юрия маточкина',
    'у.громовой': 'громовой',
    'д.давыдова': 'давыдова',
    'генерал-лейтенанта захарова': 'захарова',
    'л.иванихиной': 'иванихиной',
    'л.шевцовой': 'шевцовой',
    'з.космодемьянской': 'космодемьянской',
    'к.назаровой': 'назаровой',
}

In [11]:
second['formalname_street'] = second['formalname_street'].map(
    lambda x: street_maps.get(x, x)
)

In [12]:
df['prep_street'] = df['prep_street'].map(
    lambda x: street_maps.get(x, x)
)

In [13]:
set(df['prep_street'].unique()).difference(set(second['formalname_street'])).__len__()

40

# 2. Join

In [14]:
STR_DIGITS = [str(i) for i in range(10)]

In [15]:
def filter_number(string):
    str_number = "".join(filter(lambda x: x in STR_DIGITS, string.split('/')[0]))
    if str_number:
        return int(str_number)
    else:
        return None

def filter_number_list(string):
    return list(filter(lambda x: not(x is None), map(filter_number, string.replace('_', '-').split('-'))))

In [16]:
def get_second_info(row):
    city, settlement, street, house_number = row[['city', 'settlement', 'prep_street', 'house_number']]
    
    street_records = second[(second['formalname_city'].isin([city, settlement]))&(second['formalname_street']==street)]
    second_house_number = street_records['house_number'].unique().tolist()
    
    n_rows = street_records.shape[0]
    
    if not n_rows:
        return
    
    if pd.isna(house_number):
        second_info = (
            street_records[second_columns]
            .apply(func=lambda x: x.tolist, axis=0)
            .map(lambda x: x())
        )
    elif house_number in second_house_number:
        second_info = (
            street_records[street_records['house_number']==house_number][second_columns]
            .apply(func=lambda x: x.tolist, axis=0)
            .map(lambda x: x())
        )
    else:
        house_number = filter_number(house_number)
        map_numbers = list(map(
            filter_number_list, 
            second_house_number
        ))
        sort_map_numbers = sorted(enumerate(map_numbers), key=lambda x: min(x[1]) if x[1] else float('inf'))
        
        sim_idx = sort_map_numbers[-1][0]
        prev_idx = sort_map_numbers[0][0]
        for num, map_nums in sort_map_numbers:
            if min(map_nums) <= house_number <= max(map_nums):
                sim_idx = num
                break
            elif min(map_nums) > house_number:
                sim_idx = prev_idx
                break
            prev_idx = num
        house_number = second_house_number[prev_idx]
        second_info = (
            street_records[street_records['house_number']==house_number][second_columns]
            .apply(func=lambda x: x.tolist, axis=0)
            .map(lambda x: x())
        )

    return second_info

In [17]:
second_info = df.apply(
    get_second_info,
    axis=1
)
second_info.columns = [f"second_info_{i}" for i in second_info.columns]

In [18]:
df_second_info = df.join(second_info)

In [19]:
df_second_info.groupby('new_object')['second_info_built_year'].apply(lambda x: x.isna().mean())

new_object
False    0.138702
True     0.585780
Name: second_info_built_year, dtype: float64

In [20]:
df_second_info.to_csv('output/df_house_info_2.csv', index=False)